<a href="https://colab.research.google.com/github/mattambrogi/langchain_experiments/blob/main/langchain_api_pandas_agent_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proof of concept for allowing an agent to parse large API responses with assitance of dataframe agent


In [ ]:
!pip install langchain
!pip install openai
!pip install pandas


In [ ]:
from langchain.agents import AgentType, initialize_agent, create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.agents.agent_types import AgentType
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
import requests
import pandas as pd
from typing import Optional, Type

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    flattened = {}
    for k,v in d.items():
      new_key = f"{parent_key}{sep}{k}" if parent_key else k
      if isinstance(v, dict):
        flattened.update(flatten_dict(v, new_key, sep=sep))
      elif isinstance(v, list):
        for i, item in enumerate(v):
          if isinstance(item, dict):
            flattened.update(flatten_dict(item, f"{new_key}{sep}{i}", sep=sep))
          else:
            flattened[f"{new_key}{sep}{i}"] = item
      else:
        flattened[new_key] = v
    return flattened


In [ ]:
class GetTodosTool(BaseTool):
  """Sends a GET request to get all todos for a user"""

  name = "get_todos"
  description = "Used to get todos data for a user"

  def __init__(self):
        super().__init__()

  def _run(self, query_params = None, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
    """Use the tool"""
    response = requests.get("https://jsonplaceholder.typicode.com/todos")
    todos = response.json()
    df = pd.DataFrame(todos)
    pandas_agent = create_pandas_dataframe_agent(
        llm,
        df,
        verbose=False,
        handle_parsing_erros=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
    )
    answer = pandas_agent.run(run_manager.tags[0])
    return answer

  async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [ ]:
mock_invoice_data = [
  {"id": 1, "amount": 100, "open_date": "08/10/2023", "closed_date": "08/20/2023", "client": "John Doe"},
  {"id": 2, "amount": 200, "open_date": "08/11/2023", "closed_date": None, "client": "Jane Smith"},
  {"id": 3, "amount": 150, "open_date": "08/12/2023", "closed_date": "08/22/2023", "client": "Bob Johnson"},
  {"id": 4, "amount": 180, "open_date": "08/13/2023", "closed_date": "08/23/2023", "client": "Alice Williams"},
  {"id": 5, "amount": 220, "open_date": "08/14/2023", "closed_date": None, "client": "John Doe"},
  {"id": 6, "amount": 110, "open_date": "08/15/2023", "closed_date": "08/25/2023", "client": "Jane Smith"},
  {"id": 7, "amount": 130, "open_date": "08/16/2023", "closed_date": "08/26/2023", "client": "Bob Johnson"},
  {"id": 8, "amount": 240, "open_date": "08/17/2023", "closed_date": None, "client": "Alice Williams"},
  {"id": 9, "amount": 260, "open_date": "08/18/2023", "closed_date": "08/28/2023", "client": "John Doe"},
  {"id": 10, "amount": 210, "open_date": "08/19/2023", "closed_date": "08/29/2023", "client": "Jane Smith"},
  {"id": 11, "amount": 250, "open_date": "08/20/2023", "closed_date": "08/30/2023", "client": "Bob Johnson"},
  {"id": 12, "amount": 290, "open_date": "08/21/2023", "closed_date": None, "client": "Alice Williams"},
  {"id": 13, "amount": 270, "open_date": "08/22/2023", "closed_date": "08/32/2023", "client": "John Doe"},
  {"id": 14, "amount": 280, "open_date": "08/23/2023", "closed_date": "08/33/2023", "client": "Jane Smith"},
  {"id": 15, "amount": 320, "open_date": "08/24/2023", "closed_date": None, "client": "Bob Johnson"},
  {"id": 16, "amount": 300, "open_date": "08/25/2023", "closed_date": "08/35/2023", "client": "Alice Williams"},
  {"id": 17, "amount": 310, "open_date": "08/26/2023", "closed_date": "08/36/2023", "client": "John Doe"},
  {"id": 18, "amount": 330, "open_date": "08/27/2023", "closed_date": None, "client": "Jane Smith"},
  {"id": 19, "amount": 340, "open_date": "08/28/2023", "closed_date": "08/38/2023", "client": "Bob Johnson"},
  {"id": 20, "amount": 350, "open_date": "08/29/2023", "closed_date": "08/39/2023", "client": "Alice Williams"}
]


In [ ]:
class GetInvoicesTool(BaseTool):
  """Sends a GET request to get all invoices for a user"""

  name = "get_invoices"
  description = "Used to get invoices data for a user"

  def __init__(self):
        super().__init__()

  def _run(self, query_params = None, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
    """Use the tool"""
    response = mock_invoice_data
    #todos = response.json()
    df = pd.DataFrame(response)
    pandas_agent = create_pandas_dataframe_agent(
        llm,
        df,
        verbose=True,
        handle_parsing_erros=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
    )
    answer = pandas_agent.run(run_manager.tags[0])
    return answer

  async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
mock_transactions_data = [
  {"transaction_id": 1, "transaction_date": "08/01/2023", "transaction_type": "WITHDRAW", "transaction_amount": -100, "merchant_name": "ShopA", "mcc": "MCC1"},
  {"transaction_id": 2, "transaction_date": "08/02/2023", "transaction_type": "DEPOSIT", "transaction_amount": 150, "merchant_name": "Payroll", "mcc": "MCC2"},
  {"transaction_id": 3, "transaction_date": "08/03/2023", "transaction_type": "WITHDRAW", "transaction_amount": -120, "merchant_name": "ShopB", "mcc": "MCC3"},
  {"transaction_id": 4, "transaction_date": "08/04/2023", "transaction_type": "WITHDRAW", "transaction_amount": -130, "merchant_name": "ShopA", "mcc": "MCC1"},
  {"transaction_id": 5, "transaction_date": "08/05/2023", "transaction_type": "DEPOSIT", "transaction_amount": 200, "merchant_name": "ClientX", "mcc": "MCC4"},
  {"transaction_id": 6, "transaction_date": "08/06/2023", "transaction_type": "WITHDRAW", "transaction_amount": -80, "merchant_name": "ShopC", "mcc": "MCC5"},
  {"transaction_id": 7, "transaction_date": "08/07/2023", "transaction_type": "WITHDRAW", "transaction_amount": -60, "merchant_name": "ShopB", "mcc": "MCC3"},
  {"transaction_id": 8, "transaction_date": "08/08/2023", "transaction_type": "DEPOSIT", "transaction_amount": 100, "merchant_name": "ClientY", "mcc": "MCC4"},
  {"transaction_id": 9, "transaction_date": "08/09/2023", "transaction_type": "WITHDRAW", "transaction_amount": -90, "merchant_name": "ShopA", "mcc": "MCC1"},
  {"transaction_id": 10, "transaction_date": "08/10/2023", "transaction_type": "DEPOSIT", "transaction_amount": 175, "merchant_name": "Payroll", "mcc": "MCC2"},
  {"transaction_id": 11, "transaction_date": "08/11/2023", "transaction_type": "WITHDRAW", "transaction_amount": -50, "merchant_name": "ShopC", "mcc": "MCC5"},
  {"transaction_id": 12, "transaction_date": "08/12/2023", "transaction_type": "DEPOSIT", "transaction_amount": 180, "merchant_name": "ClientZ", "mcc": "MCC4"},
  {"transaction_id": 13, "transaction_date": "08/13/2023", "transaction_type": "WITHDRAW", "transaction_amount": -70, "merchant_name": "ShopB", "mcc": "MCC3"},
  {"transaction_id": 14, "transaction_date": "08/14/2023", "transaction_type": "WITHDRAW", "transaction_amount": -200, "merchant_name": "ShopA", "mcc": "MCC1"},
  {"transaction_id": 15, "transaction_date": "08/15/2023", "transaction_type": "DEPOSIT", "transaction_amount": 160, "merchant_name": "ClientX", "mcc": "MCC4"},
  {"transaction_id": 16, "transaction_date": "08/16/2023", "transaction_type": "WITHDRAW", "transaction_amount": -110, "merchant_name": "ShopC", "mcc": "MCC5"},
  {"transaction_id": 17, "transaction_date": "08/17/2023", "transaction_type": "DEPOSIT", "transaction_amount": 150, "merchant_name": "Payroll", "mcc": "MCC2"},
  {"transaction_id": 18, "transaction_date": "08/18/2023", "transaction_type": "WITHDRAW", "transaction_amount": -100, "merchant_name": "ShopB", "mcc": "MCC3"},
  {"transaction_id": 19, "transaction_date": "08/19/2023", "transaction_type": "WITHDRAW", "transaction_amount": -90, "merchant_name": "ShopC", "mcc": "MCC5"},
  {"transaction_id": 20, "transaction_date": "08/20/2023", "transaction_type": "DEPOSIT", "transaction_amount": 180, "merchant_name": "ClientY", "mcc": "MCC4"}
]


In [ ]:
class GetTransactionsTool(BaseTool):
  """Sends a GET request to get all bank transactions for a user. Does not require input."""

  name = "get_transactions"
  description = "Used to get bank transactions data for a user. Can answer general questions about spending. Does not require input."

  def __init__(self):
        super().__init__()

  def _run(self, query_params = None, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
    """Use the tool"""
    response = mock_transactions_data
    #todos = response.json()
    df = pd.DataFrame(response)
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%m/%d/%Y')
    pandas_agent = create_pandas_dataframe_agent(
        llm,
        df,
        verbose=True,
        handle_parsing_erros=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
    )
    answer = pandas_agent.run(run_manager.tags[0])
    return answer

  async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
tools = [GetTodosTool(), GetInvoicesTool(), GetTransactionsTool()]

In [ ]:
agent = initialize_agent(
    llm=llm,
    tools=tools,
    handle_parsing_errors=True,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)


In [ ]:
query = "How many todos does user 1 have and what is the title of their most recent one?"

agent.run(
    query,
    tags = [query]
)



> Entering new AgentExecutor chain...
I need to use the get_todos tool to get the todos data for user 1. Then I can count the number of todos and find the most recent one.
Action: get_todos
Action Input: user_id = 1

> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['userId'] == 1]['id'].count()"}`


20
Invoking: `python_repl_ast` with `{'query': "df[df['userId'] == 1].sort_values('id', ascending=False)['title'].iloc[0]"}`


ullam nobis libero sapiente ad optio sintUser 1 has 20 todos and the title of their most recent one is "ullam nobis libero sapiente ad optio sint".

> Finished chain.

Observation: User 1 has 20 todos and the title of their most recent one is "ullam nobis libero sapiente ad optio sint".
Thought:I now know the final answer
Final Answer: User 1 has 20 todos and the title of their most recent one is "ullam nobis libero sapiente ad optio sint".

> Finished chain.


'User 1 has 20 todos and the title of their most recent one is "ullam nobis libero sapiente ad optio sint".'

In [ ]:
query = "How much is my oldest open invoice for, and who is the client?"

answer = agent.run(
    query,
    tags = [query]
)

print(answer)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "oldest_open_invoice = df[df['closed_date'].isnull()].sort_values('open_date').iloc[0]\namount = oldest_open_invoice['amount']\nclient = oldest_open_invoice['client']\namount, client"}`


(200, 'Jane Smith')Your oldest open invoice is for $200, and the client is Jane Smith.

> Finished chain.
Your oldest open invoice is for $200, and the client is Jane Smith.
